In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [15]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for KNN
param_grid = {'n_neighbors': [1, 3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}

# Create a KNN classifier
knn_classifier = KNeighborsClassifier()

# Create a GridSearchCV object for KNN
grid = GridSearchCV(knn_classifier, param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fit the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [1, 3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [16]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}: 0.9183922682195829


In [18]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [ 4 45]]


In [19]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [20]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
#grid.predict_proba(X_test): This part of the code generates the predicted probabilities of the positive class for each instance in the test set (X_test).
#[:,1] = [row:column] this line indicates entire row with 1 st column only.

0.9411764705882354

In [21]:
table=pd.DataFrame.from_dict(re)

In [22]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008667,0.006739,0.016592,0.007691,euclidean,1,uniform,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",0.847141,0.866968,0.815056,0.813179,0.923510,0.853171,0.040562,19
1,0.004182,0.004097,0.005616,0.005758,euclidean,1,distance,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",0.847141,0.866968,0.815056,0.813179,0.923510,0.853171,0.040562,19
2,0.002805,0.005609,0.023424,0.010930,euclidean,3,uniform,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei...",0.867478,0.906252,0.869709,0.962636,0.924528,0.906121,0.035642,2
3,0.002909,0.003965,0.007260,0.004536,euclidean,3,distance,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei...",0.847141,0.906252,0.814409,0.906935,0.924528,0.879853,0.041903,15
4,0.001774,0.002248,0.015173,0.003558,euclidean,5,uniform,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei...",0.867478,0.906252,0.869709,0.944161,0.981233,0.913767,0.043854,1
5,0.006258,0.007664,0.010019,0.004914,euclidean,5,distance,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei...",0.847141,0.886792,0.851527,0.944161,0.962264,0.898377,0.047185,10
6,0.003210,0.003944,0.015330,0.004412,euclidean,7,uniform,"{'metric': 'euclidean', 'n_neighbors': 7, 'wei...",0.867478,0.886792,0.851527,0.944161,0.962264,0.902445,0.043310,4
7,0.002067,0.004134,0.007026,0.004921,euclidean,7,distance,"{'metric': 'euclidean', 'n_neighbors': 7, 'wei...",0.847141,0.906252,0.851527,0.944161,0.943041,0.898425,0.042363,9
8,0.003124,0.006248,0.015215,0.002918,euclidean,9,uniform,"{'metric': 'euclidean', 'n_neighbors': 9, 'wei...",0.847141,0.886792,0.850543,0.944161,0.962264,0.898180,0.047381,11
9,0.001114,0.001375,0.006628,0.008117,euclidean,9,distance,"{'metric': 'euclidean', 'n_neighbors': 9, 'wei...",0.847141,0.886792,0.851527,0.944161,0.981233,0.902171,0.052618,6


In [23]:
Future_Prediction=grid.predict([[30,1000,1]])
# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
